# パイプラインを公開する

[前のラボ](labdocs/Lab06A.md)で、パイプラインを作成しました。次に、サービスとして公開します。

## ワークスペースに接続する

最初に行う必要があるのは、Azure ML SDKを使用してワークスペースに接続することです。

> **Note**: 前の演習を完了してから、Azureサブスクリプションとの認証済みセッションの有効期限が切れた場合、再認証するように求められます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## パイプラインを公開する

パイプラインを作成してテストしたら、RESTサービスとして公開できます。 [前のラボ](labdocs/Lab06A.md)でパイプラインを実行したため、その実行への参照を取得し、それを使用してパイプラインを公開できます。

In [ ]:
# Get the most recent run of the pipeline
experiment_name = 'diabetes-training-pipeline'
pipeline_experiment = ws.experiments.get(experiment_name)
pipeline_run = list(pipeline_experiment.get_runs())[0]

# Publish the pipeline from the run
published_pipeline = pipeline_run.publish_pipeline(
    name="Diabetes_Training_Pipeline", description="Trains diabetes model", version="1.0")

published_pipeline

公開されたパイプラインにはエンドポイントがあり、[Azure Machine Learning studio](https://ml.azure.com)の**Endpoints**ページ（**Pipeline Endpoints**タブ）で確認できます。 また、公開されたパイプラインオブジェクトのプロパティとしてそのURIを見つけることもできます。:

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

## パイプラインエンドポイントを使用する

エンドポイントを使用するには、クライアントアプリケーションがHTTP経由でREST呼び出しを行う必要があります。この要求は認証される必要があるため、認証ヘッダーが必要です。実際のアプリケーションでは、認証に使用するサービスプリンシパルが必要ですが、これをテストするには、現在の接続からAzureワークスペースへの認証ヘッダーを使用します。これは、次のコードを使用して取得できます:

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print(auth_header)

これで、RESTインターフェースを呼び出す準備ができました。パイプラインは非同期に実行されるため、識別子を取得します。これを使用して、実行中のパイプラインの実験を追跡できます:

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

実行IDがあるため、**RunDetails**ウィジェットを使用して、実行中の実験を表示できます。

> **Note**: 各ステップは出力の再利用を許可するように構成されているため、パイプラインは迅速に完了するはずです。これは、主に便宜上、このコースの時間を節約するために行われました。実際には、データが変更された場合に備えて毎回最初のステップを実行し、ステップ1からの出力が変更された場合にのみ後続のステップをトリガーします。

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
RunDetails(published_pipeline_run).show()

## パイプラインをスケジュールする

糖尿病患者の診療所が毎週新しいデータを収集し、それをデータセットに追加するとします。毎週パイプラインを実行して、新しいデータでモデルを再トレーニングできます。

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

# Submit the Pipeline every Monday at 00:00 UTC
recurrence = ScheduleRecurrence(frequency="Week", interval=1, week_days=["Monday"], time_of_day="00:00")
weekly_schedule = Schedule.create(ws, name="weekly-diabetes-training", 
                                  description="Based on time",
                                  pipeline_id=published_pipeline.id, 
                                  experiment_name=experiment_name, 
                                  recurrence=recurrence)

このようなワークスペースで定義されているスケジュールを取得できます:

In [ ]:
schedules = Schedule.list(ws)
schedules

このように最新の実行を確認できます:

In [ ]:
pipeline_experiment = ws.experiments.get(experiment_name)
latest_run = list(pipeline_experiment.get_runs())[0]

latest_run.get_details()

> **詳しくは**: [ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-schedule-pipelines)でパイプラインのスケジュールの詳細を確認できます。